In [29]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_predict

X, y = make_classification(n_samples=1000, n_features=10, n_informative=3, n_classes=2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)


sgdc_clf = SGDClassifier(random_state=42)
sgdc_clf.fit(X_train, y_train)

y_train_pred = cross_val_predict(sgdc_clf, X_train, y_train, cv=3)

cm = confusion_matrix(y_train, y_train_pred)

conf_matrix = pd.DataFrame(cm, index=["Actual Class 0", "Actual Class 1"], columns=["Predicted Class 0", "Predicted Class 1"])

display(conf_matrix)
print("Score du recall : ", recall_score(y_train, y_train_pred))
print("Score de la precision : ", precision_score(y_train, y_train_pred))
print("Score F1 : ", f1_score(y_train, y_train_pred))

y_scores = cross_val_predict(sgdc_clf, y_train, y_train_pred)

,Predicted Class 0,Predicted Class 1
Actual Class 0,332,71
Actual Class 1,29,368


Score du recall :  0.9269521410579346
Score de la precision :  0.8382687927107062
Score F1 :  0.8803827751196173
